In [14]:
import pandas as pd
from sqlalchemy import create_engine
import sqlite3
from sqlalchemy.types import Numeric, Integer, String


DB = {'servername': 'XPS15\\SQLEXPRESS', 'database': 'gecombineerde_database2'}

#SQLAlchemy engine voor SQL Server
engine_str = (
    f"mssql+pyodbc://{DB['servername']}/{DB['database']}?"
    "trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server"
)

engine = create_engine(engine_str)

In [15]:
# Lijst met SQLite-bestanden
sqlite_files = ['go_crm_train.sqlite', 'go_sales_train.sqlite', 'go_staff_train.sqlite']

# Dictionaries om per tabel:
# - de union van kolommen (als set) en
# - de lijst van dataframes uit de verschillende SQLite-bestanden op te slaan
table_columns = {}  # key: tabelnaam, value: set met kolomnamen
dataframes = {}     # key: tabelnaam, value: lijst met DataFrames

In [16]:
# Loop door alle SQLite-bestanden
for file in sqlite_files:
    conn = sqlite3.connect(file)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables = cursor.fetchall()

    for (table_name,) in tables:
        # Lees de tabel in een DataFrame
        df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)

        # Als deze tabel nog niet eerder is tegengekomen, initialiseer de dictionaries
        if table_name not in table_columns:
            table_columns[table_name] = set(df.columns)
            dataframes[table_name] = [df]
        else:
            # Voeg de kolommen van de huidige DataFrame toe aan de union van kolommen
            table_columns[table_name].update(df.columns)
            dataframes[table_name].append(df)

    conn.close()

In [18]:
# Voor elke tabel: combineer de DataFrames zodat alle kolommen aanwezig zijn
for table_name, df_list in dataframes.items():
    # Haal de union van alle kolommen op
    union_cols = list(table_columns[table_name])

    # Herindexeer iedere DataFrame zodat alle kolommen aanwezig zijn (ontbrekende waarden = NaN)
    df_list = [df.reindex(columns=union_cols) for df in df_list]

    # Definieer een mapping van kolomnamen naar SQLAlchemy datatypes.
    # Pas de kolomnamen en types aan op basis van jouw data.
    dtype_mapping = {
        'prijs': Numeric(10, 2),       # Numeriek met precisie 10 en schaal 2
        'aantal': Integer(),           # Integer
        'beschrijving': String(255)    # String met max 255 tekens
    }

    # Combineer de DataFrames
    merged_df = pd.concat(df_list, ignore_index=True)

    # Schrijf de gecombineerde DataFrame weg naar SQL Server.
    # if_exists='replace' maakt de tabel aan (of vervangt hem als die al bestaat)
    merged_df.to_sql(table_name, engine, if_exists='replace', index=False, dtype=dtype_mapping)
    print(f"Tabel '{table_name}' is naar SQL Server gestuurd.")


Tabel 'age_group' is naar SQL Server gestuurd.
Tabel 'country' is naar SQL Server gestuurd.
Tabel 'retailer_segment' is naar SQL Server gestuurd.
Tabel 'sales_territory' is naar SQL Server gestuurd.
Tabel 'retailer' is naar SQL Server gestuurd.
Tabel 'retailer_site' is naar SQL Server gestuurd.
Tabel 'retailer_type' is naar SQL Server gestuurd.
Tabel 'sales_demographic' is naar SQL Server gestuurd.
Tabel 'retailer_headquarters' is naar SQL Server gestuurd.
Tabel 'retailer_contact' is naar SQL Server gestuurd.
Tabel 'order_details' is naar SQL Server gestuurd.
Tabel 'order_header' is naar SQL Server gestuurd.
Tabel 'order_method' is naar SQL Server gestuurd.
Tabel 'product_line' is naar SQL Server gestuurd.
Tabel 'product_type' is naar SQL Server gestuurd.
Tabel 'product' is naar SQL Server gestuurd.
Tabel 'return_reason' is naar SQL Server gestuurd.
Tabel 'returned_item' is naar SQL Server gestuurd.
Tabel 'sales_branch' is naar SQL Server gestuurd.
Tabel 'sales_staff' is naar SQL Serve